In [1]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

def read_csv(root_path):
    with open(root_path + 'driving_log.csv', 'r') as f:
        reader = csv.reader(f)
        for line in reader:
            yield line

def make_local_path(path, local_root):
    basename = path.split('/')[-1]
    return local_root + 'IMG/' + basename

def load_image(local_path):
    return cv2.imread(local_path)

def extract_images(csv_line, local_root):
    paths = csv_line[0:3]
    return [load_image(make_local_path(p, local_root)) for p in paths]

def extract_measurements(csv_line):
    return float(csv_line[3])

In [2]:
images = []
measurements = []

forward_img = './data/forward/'
reverse_img = './data/reverse/'

print("Reading forward images...\n")
for line in read_csv(forward_img):
    images.append(extract_images(line, forward_img)[0])
    measurements.append(extract_measurements(line))

print("Reading reverse images...\n")
for line in read_csv(reverse_img):
    images.append(extract_images(line, reverse_img)[0])
    measurements.append(extract_measurements(line))

X_train = np.array(images, np.float32)
y_train = np.array(measurements, np.float32)

print("Training set shape: ", X_train.shape)
print("Labels shape: ", y_train.shape)


Reading forward images...

Reading reverse images...

Training set shape:  (15911, 160, 320, 3)
Labels shape:  (15911,)


In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, MaxPooling2D, Lambda, Cropping2D, Dropout

image_shape = X_train.shape[1:]
normalize = lambda x: (x - 127.0) / 127.0
cropping = ((60, 0), (0, 0))
dropout_rate = 0.5

model = Sequential()
model.add(Lambda(normalize, input_shape=image_shape))
model.add(Cropping2D(cropping=cropping))
model.add(Convolution2D(24, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
model.add(Convolution2D(36, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
model.add(Convolution2D(48, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
model.add(Convolution2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(Convolution2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))

model.add(Flatten())
model.add(Dropout(dropout_rate))
model.add(Dense(100, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(50, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, validation_split=0.2, shuffle=True, verbose=2, batch_size=256, epochs=10)

model.save('model.h5')

Train on 12728 samples, validate on 3183 samples
Epoch 1/10
22s - loss: 0.0568 - val_loss: 0.0536
Epoch 2/10
21s - loss: 0.0530 - val_loss: 0.0500
Epoch 3/10
21s - loss: 0.0513 - val_loss: 0.0489
Epoch 4/10
21s - loss: 0.0503 - val_loss: 0.0485
Epoch 5/10
21s - loss: 0.0495 - val_loss: 0.0473
Epoch 6/10
21s - loss: 0.0497 - val_loss: 0.0492
Epoch 7/10
21s - loss: 0.0490 - val_loss: 0.0486
Epoch 8/10
21s - loss: 0.0482 - val_loss: 0.0484
Epoch 9/10
21s - loss: 0.0477 - val_loss: 0.0482
Epoch 10/10
21s - loss: 0.0471 - val_loss: 0.0484


In [10]:
model.layers[4].output_shape

(None, 9, 37, 48)

In [ ]:
epochs = history.epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.ylim(0, 0.2)